In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re

In [2]:
url = """https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/"""

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
soup=BeautifulSoup(page.content, 'html.parser')

In [5]:
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
title

'Deep learning impact on areas of e-learning?'

In [6]:
content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
content

' eLearning as technology becomes more affordable in higher education but having a big barrier in the cost of developing its resources. Deep learning using artificial intelligence continues to become more and more popular and having impacts on many areas of eLearning. It offers online learners of the future intuitive algorithms and automated delivery of eLearning content through modern LMS platforms. This paper aims to survey various applications of deep learning approaches for developing the resources of the eLearning platform, in which predictions, algorithms, and analytics come together to create more personalized future eLearning experiences. In addition, deep learning models for developing the contents of the eLearning platform, deep learning framework that enable deep learning systems into eLearning and its development, benefits & future trends of deep learning in eLearning, the relevant deep learning-based artificial intelligence tools, and a platform enabling the developer and 

In [7]:
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
content

' eLearning as technology becomes more affordable in higher education but having a big barrier in the cost of developing its resources Deep learning using artificial intelligence continues to become more and more popular and having impacts on many areas of eLearning It offers online learners of the future intuitive algorithms and automated delivery of eLearning content through modern LMS platforms This paper aims to survey various applications of deep learning approaches for developing the resources of the eLearning platform in which predictions algorithms and analytics come together to create more personalized future eLearning experiences In addition deep learning models for developing the contents of the eLearning platform deep learning framework that enable deep learning systems into eLearning and its development benefits  future trends of deep learning in eLearning the relevant deep learningbased artificial intelligence tools and a platform enabling the developer and learners to qu

In [8]:
text = content.split()
text

['eLearning',
 'as',
 'technology',
 'becomes',
 'more',
 'affordable',
 'in',
 'higher',
 'education',
 'but',
 'having',
 'a',
 'big',
 'barrier',
 'in',
 'the',
 'cost',
 'of',
 'developing',
 'its',
 'resources',
 'Deep',
 'learning',
 'using',
 'artificial',
 'intelligence',
 'continues',
 'to',
 'become',
 'more',
 'and',
 'more',
 'popular',
 'and',
 'having',
 'impacts',
 'on',
 'many',
 'areas',
 'of',
 'eLearning',
 'It',
 'offers',
 'online',
 'learners',
 'of',
 'the',
 'future',
 'intuitive',
 'algorithms',
 'and',
 'automated',
 'delivery',
 'of',
 'eLearning',
 'content',
 'through',
 'modern',
 'LMS',
 'platforms',
 'This',
 'paper',
 'aims',
 'to',
 'survey',
 'various',
 'applications',
 'of',
 'deep',
 'learning',
 'approaches',
 'for',
 'developing',
 'the',
 'resources',
 'of',
 'the',
 'eLearning',
 'platform',
 'in',
 'which',
 'predictions',
 'algorithms',
 'and',
 'analytics',
 'come',
 'together',
 'to',
 'create',
 'more',
 'personalized',
 'future',
 'eLearn

In [9]:
len(text)

2155

In [10]:
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

76


In [11]:
#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)

40


In [16]:
filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])

# Adding Subjectivity & Polarity
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)

data

,url,title,content,filter_content,Positive_Score,Negative_Score,polarity,subjectivity
0,https://insights.blackcoffer.com/deep-learning...,Deep learning impact on areas of e-learning?,eLearning as technology becomes more affordab...,eLearning as technology becomes more affordabl...,76,40,0.168589,0.531156


In [17]:
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)

Word average = 10664.0


In [18]:
import textstat

In [19]:
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print(FOG_INDEX)

865.99


In [20]:
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
AVG_NUMBER_OF_WORDS_PER_SENTENCE=print(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))

2155.0


In [21]:
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print(COMPLEX_WORDS)

3898


In [22]:
#Word Count
Word_Count=len(content)
print(Word_Count)

12822


In [23]:
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print(pcw)

30.40087349867415


In [24]:
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print(Average_Word_Length)

4.948491879350348


In [25]:
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
print(syllable_count/len(content.split()))

The AVG number of syllables in the word is: 
2.047795823665893


In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re
url = """https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/"""
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
# title

content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
# print(content)
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
# print(content)
text = content.split()
# print(text)
len(text)
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)


filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
print(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
syllable = (syllable_count/len(content.split()))
print(syllable)

data = [[url,title,content,filter_content,Positive_score,Negative_score,AVG_SENTENCE_LENGTH,pcw,FOG_INDEX,
         AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score","Avg_Sentence_Length"
                               ,"Percentage_Complex_Word","Fog_Index"," AVG_NUMBER_OF_WORDS_PER_SENTENCE","COMPLEX_WORDS",
                               "Word_Count","syllable","Average_Word_Length"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data



76
40
Word average = 10664.0
FOG INDEX =  865.99
Average no of words per sentence
2155.0
Complex Words 3898
Word Count 12822
Percentage of Complex Words 30.40087349867415
Average Word per Length 4.948491879350348
The AVG number of syllables in the word is: 
2.047795823665893


,url,title,content,filter_content,Positive_Score,Negative_Score,Avg_Sentence_Length,Percentage_Complex_Word,Fog_Index,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length,polarity,subjectivity
0,https://insights.blackcoffer.com/deep-learning...,Deep learning impact on areas of e-learning?,eLearning as technology becomes more affordab...,eLearning as technology becomes more affordabl...,76,40,10664.0,30.400873,865.99,2155.0,3898,12822,2.047796,4.948492,0.168589,0.531156


In [2]:
data.to_csv(r'C:\Users\Om Bhandwalkar\Desktop\BlackCoffer Assignment\Output\url_46.csv')